# Ellipse

The notebook contains demonstrations of ellipse-line intersection and ellipse tangent point calculation algorithms used in constructing the target for the keypoint model.

In [ ]:
import cv2
import numpy as np
from ellipse import LsqEllipse
from scipy.optimize import curve_fit
from src.models.hrnet.dataset import HRNetDataset
from src.datatools.geom import find_longest_line
from src.datatools.ellipse import ellipse_line_intersect, find_tangent_point
import matplotlib.pyplot as plt
from matplotlib.patches import Ellipse

%load_ext autoreload
%autoreload 2

### Load the data

In [ ]:
dataset_path = '/workdir/data/dataset/valid/'
STRIDE = 8
SIGMA = 7
IMG_SIZE = (540, 960)
dataset = HRNetDataset(dataset_path, transform=None)

In [ ]:
sample = dataset[30]
annot = sample['raw_annot']
img = sample['image'][:, :, ::-1]
print(annot)
plt.imshow(img)

In [ ]:
circ_x = [s['x'] for s in annot['Circle central']]
circ_y = [s['y'] for s in annot['Circle central']]
line_x = [s['x'] for s in annot['Middle line']]
line_y = [s['y'] for s in annot['Middle line']]

### Line-ellipse intersection

In [ ]:

conic = np.array(list(zip(circ_x, circ_y)))
line = np.array(list(zip(line_x, line_y)))

reg = LsqEllipse().fit(conic)
conic_coeffs = reg.coefficients
a, b = np.polyfit(line[:, 0], line[:, 1], 1)

res = ellipse_line_intersect(conic_coeffs, line)
print('Line-ellipse intersection coordinates:\n', res)

In [ ]:
center, width, height, phi = reg.as_parameters()
fig = plt.figure(figsize=(6, 6))
ax = plt.subplot()
plt.gca().invert_yaxis()
ax.plot(circ_x, circ_y, 'ro', zorder=1)  #  Red points - The points from the annotation
ellipse = Ellipse(
        xy=center, width=2*width, height=2*height, angle=np.rad2deg(phi),
        edgecolor='b', fc='None', lw=2, label='Fit', zorder=2
    )  # Blue line - the fitted ellipse
ax.add_patch(ellipse)
ax.plot(line_x, line_y, color='k', linestyle='-', linewidth=2)  # The black line
ax.plot(center[0], center[1], 'bo')  # Blue point - the calculated center of the ellipse
ax.plot(res[:, 0], res[:, 1], 'rx')  # Red crosses - calculated intersection points of the line and the ellipse

plt.xlabel('$X$')
plt.ylabel('$Y$')
plt.legend()
plt.show()

In [ ]:
# The line - ellipse intersection point visualization on the original image
img_vis = img.copy()
for point in res:
    point = [int(np.round(p)) for p in point]
    cv2.circle(img_vis, point, 3, (255, 0, 0), -1)
plt.imshow(img_vis)

### Tangent points

In [ ]:
point = (line_x[0], line_y[0])
# Find coordinates of two tangent points
px0, py0 = find_tangent_point(conic_coeffs, point, 0)
px1, py1 = find_tangent_point(conic_coeffs, point, 1)
x0, y0 = point
tg_x = (px0, x0, px1)
tg_y = (py0, y0, py1)

In [ ]:
center, width, height, phi = reg.as_parameters()
fig = plt.figure(figsize=(6, 6), dpi=200)
ax = plt.subplot()
ax.axis('equal')
ax.imshow(img.copy())
ax.plot(circ_x, circ_y, 'ro', zorder=1, markersize=3)  # Red dots - ellipse points from the annotation

ellipse = Ellipse(
        xy=center, width=2*width, height=2*height, angle=np.rad2deg(phi),
        fc='None', lw=1, label='Fit', zorder=1, color='brown'
    )  # Brown (dark red) line - the ellipse fit
ax.add_patch(ellipse)
ax.plot(line_x, line_y, color='b', linestyle='-', linewidth=1)  # The blue line
ax.plot(tg_x, tg_y, color='purple', linestyle='-', linewidth=0.5)  # Purple lines - the tangent lines
ax.plot(res[:, 0], res[:, 1], 'gx')  # Gren crosses - the calculated line-ellipse intersection points
ax.plot(tg_x[::2], tg_y[::2], 'purple', marker='+', linestyle='none')  # Purple pluses - the calculated tangent points

plt.xlabel('$X$')
plt.ylabel('$Y$')
plt.legend()
plt.show()